In [48]:
from Bio import SeqIO
from Bio import Entrez

Entrez.email = "vasyl.haievyi@student.uj.edu.pl"

handle = Entrez.efetch(db='protein', id='40886941', rettype='gb', retmode='text')
record = SeqIO.read(handle, 'genbank')
human_hemo = record.seq

handle = Entrez.efetch(db='protein', id='34849618', rettype='gb', retmode='text')
record = SeqIO.read(handle, 'genbank')
rat_hemo = record.seq

print(human_hemo)
print(rat_hemo)

MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRLFESFGDLFTPDAVMGNPKVKAHGKKVLGAFSDGPAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
MVHLTDAEKAAVNGLWGKVNPDDVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNPKVKAHGKKVINAFNDGLKHLDNLKGTFAHLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKEFTPCAQAAFQKVVAGVASALAHKYH


In [49]:
print(len(human_hemo))
print(len(rat_hemo))

147
147


In [50]:
from Bio.SubsMat import MatrixInfo as mi
blosum62 = mi.blosum62

In [51]:
import numpy as np
NW_table = np.zeros((len(human_hemo) + 1, len(rat_hemo) + 1))
value_origin = [[[] for _ in range(len(rat_hemo) + 1)] for _ in range(len(human_hemo) + 1)]

In [52]:
break_penalty = -7

for row in range(1, len(human_hemo) + 1):
    NW_table[row][0] = break_penalty * row

for col in range(1, len(rat_hemo) + 1):
    NW_table[0][col] = break_penalty * col   

In [53]:
NW_table

array([[    0.,    -7.,   -14., ..., -1015., -1022., -1029.],
       [   -7.,     0.,     0., ...,     0.,     0.,     0.],
       [  -14.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [-1015.,     0.,     0., ...,     0.,     0.,     0.],
       [-1022.,     0.,     0., ...,     0.,     0.,     0.],
       [-1029.,     0.,     0., ...,     0.,     0.,     0.]])

In [54]:
def get_blosum62_penalty(acid1, acid2) -> int:
    if (acid1, rat_hemo[col - 1]) in blosum62:
        return blosum62[(acid1, acid2)]
    else:
        return blosum62[(acid2, acid1)]

def NW_table_possible_values():
    blosum62_penalty = get_blosum62_penalty(human_hemo[row - 1], rat_hemo[col - 1])

    match_or_subs = NW_table[row - 1][col - 1] + blosum62_penalty
    human_break = NW_table[row - 1][col] + break_penalty
    rat_break = NW_table[row][col - 1] + break_penalty

    return {"match_or_subs" : match_or_subs, "human_break" : human_break, "rat_break" : rat_break }

def populate_path_matrix(row, col ,possible_values: dict):
    if NW_table[row][col] == possible_values["match_or_subs"]:
        value_origin[row][col].append((row - 1, col - 1))
    if NW_table[row][col] == possible_values["human_break"]:
        value_origin[row][col].append((row - 1, col))
    if NW_table[row][col] == possible_values["rat_break"]:
        value_origin[row][col].append((row - 1, col))

In [55]:
for row in range(1, len(human_hemo) + 1):
    for col in range(1, len(rat_hemo) + 1):
        possible_values = NW_table_possible_values()

        NW_table[row][col] = max(possible_values.values())

        populate_path_matrix(row, col, possible_values)

In [56]:
NW_table

array([[    0.,    -7.,   -14., ..., -1015., -1022., -1029.],
       [   -7.,     5.,    -2., ..., -1003., -1010., -1017.],
       [  -14.,    -2.,     9., ...,  -992.,  -999., -1006.],
       ...,
       [-1015., -1003.,  -992., ...,   617.,   610.,   603.],
       [-1022., -1010.,  -999., ...,   610.,   624.,   617.],
       [-1029., -1017., -1006., ...,   603.,   617.,   632.]])

In [57]:
row, col = len(human_hemo), len(rat_hemo)

def print_paths(row, col, current_path=("", "")):
    if (row, col) == (0, 0):
        connection = ""
        for (upper_acid, lower_acid) in zip(*current_path):
            connection +=   "|" if upper_acid == lower_acid else " " 

        symbols_in_line = 80

        for idx in range(0, len(connection), symbols_in_line):
            print(current_path[0][idx: idx + symbols_in_line])
            print(connection[idx: idx + symbols_in_line])
            print(current_path[1][idx: idx + symbols_in_line])
            print()
        print("=" * symbols_in_line)
    else:
        coords = value_origin[row][col]
        for coord in coords:
            if coord == (row - 1, col - 1):
                print_paths(row - 1, col - 1, (human_hemo[row - 1] + current_path[0] , rat_hemo[col - 1] + current_path[1]))
            elif coord == (row, col - 1):
                print_paths(row, col - 1, ("-" + current_path[0] , rat_hemo[col - 1] + current_path[1]))
            elif coord == (row - 1, col):
                print_paths(row - 1, col, (human_hemo[row - 1] + current_path[0] , "-" + current_path[1])) 

print_paths(row, col)

MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRLFESFGDLFTPDAVMGNPKVKAHGKKVLGAFSDGPAHLD
|||||  || ||  |||||| | |||||||||||||||||| | |||||    | |||||||||||||  || ||  |||
MVHLTDAEKAAVNGLWGKVNPDDVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNPKVKAHGKKVINAFNDGLKHLD

NLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
||||||| |||||||||||||||||||||  | || || ||||||  ||| |||||||| |||||||
NLKGTFAHLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKEFTPCAQAAFQKVVAGVASALAHKYH



In [58]:
from Bio import pairwise2

alignments = pairwise2.align.globalds(human_hemo, rat_hemo, match_dict=blosum62, open=-7, extend=-7)

print(len(alignments))

print(*alignments[0][:2])

1
MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRLFESFGDLFTPDAVMGNPKVKAHGKKVLGAFSDGPAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH MVHLTDAEKAAVNGLWGKVNPDDVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNPKVKAHGKKVINAFNDGLKHLDNLKGTFAHLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKEFTPCAQAAFQKVVAGVASALAHKYH
